In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain.prompts import PipelinePromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
fullTemplate="""{introduction}

{example}

{start}"""

In [5]:
fullPrompt=PromptTemplate.from_template(template=fullTemplate)

In [ ]:
fullPrompt

PromptTemplate(input_variables=['example', 'introduction', 'start'], input_types={}, partial_variables={}, template='{introduction}\n\n{example}\n\n{start}')

In [7]:
introductionTemplate="""You are impersonating {person}."""
introductionPrompt=PromptTemplate.from_template(template=introductionTemplate)

In [8]:
introductionPrompt.format(person="Elon")

'You are impersonating Elon.'

In [9]:
exampleTemplate="""
Here's an example of Interaction:
Q: {exampleQ}
A: {exampleA}
"""
examplePrompt=PromptTemplate.from_template(template=exampleTemplate)

In [11]:
print(exampleTemplate.format(exampleQ="What's your Favourite Car",exampleA="Tesla"))


Here's an example of Interaction:
Q: What's your Favourite Car
A: Tesla



In [12]:
startTemplate="""Now, do this for real!

Q:{input}
A:
"""
startPrompt=PromptTemplate.from_template(template=startTemplate)

In [13]:
startTemplate.format(input="What's his Social Media Handle?")

"Now, do this for real!\n\nQ:What's his Social Media Handle?\nA:\n"

In [14]:
inputPrompt=[
    ("introduction",introductionPrompt),
    ("example",examplePrompt),
    ("start",startPrompt)
]

In [15]:
inputPrompt

[('introduction',
  PromptTemplate(input_variables=['person'], input_types={}, partial_variables={}, template='You are impersonating {person}.')),
 ('example',
  PromptTemplate(input_variables=['exampleA', 'exampleQ'], input_types={}, partial_variables={}, template="\nHere's an example of Interaction:\nQ: {exampleQ}\nA: {exampleA}\n")),
 ('start',
  PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='Now, do this for real!\n\nQ:{input}\nA:\n'))]

In [16]:
pipelinePrompt=PipelinePromptTemplate(final_prompt=fullPrompt,pipeline_prompts=inputPrompt)

C:\Users\MSI\AppData\Local\Temp\ipykernel_6656\2211780879.py:1: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipelinePrompt=PipelinePromptTemplate(final_prompt=fullPrompt,pipeline_prompts=inputPrompt)


In [17]:
pipelinePrompt

PipelinePromptTemplate(input_variables=['person', 'input', 'exampleQ', 'exampleA'], input_types={}, partial_variables={}, final_prompt=PromptTemplate(input_variables=['example', 'introduction', 'start'], input_types={}, partial_variables={}, template='{introduction}\n\n{example}\n\n{start}'), pipeline_prompts=[('introduction', PromptTemplate(input_variables=['person'], input_types={}, partial_variables={}, template='You are impersonating {person}.')), ('example', PromptTemplate(input_variables=['exampleA', 'exampleQ'], input_types={}, partial_variables={}, template="\nHere's an example of Interaction:\nQ: {exampleQ}\nA: {exampleA}\n")), ('start', PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='Now, do this for real!\n\nQ:{input}\nA:\n'))])

In [18]:
pipelinePrompt.input_variables

['person', 'input', 'exampleQ', 'exampleA']

In [19]:
print(pipelinePrompt.format(
    person="Elon Mask",
    exampleQ="What's your favourite Car?",
    exampleA="Tesla",
    input="What's your Social Media Handle?"
))

You are impersonating Elon Mask.


Here's an example of Interaction:
Q: What's your favourite Car?
A: Tesla


Now, do this for real!

Q:What's your Social Media Handle?
A:



In [20]:
chain=RunnablePassthrough()|pipelinePrompt|llmGemini|StrOutputParser()|RunnableLambda(lambda s:s.strip())

In [21]:
chain.invoke(input={'exampleQ':"What's your favourite Car?","exampleA":"Tesla","person":"Elon Musk","input":"What's his other companies?"})

'SpaceX, Neuralink, Boring Company, xAI... and X, of course. Gotta love X.'

In [22]:
chain.get_graph().print_ascii()

   +------------------+    
   | PassthroughInput |    
   +------------------+    
             *             
             *             
             *             
      +-------------+      
      | Passthrough |      
      +-------------+      
             *             
             *             
             *             
+------------------------+ 
| PipelinePromptTemplate | 
+------------------------+ 
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
        +--------+         
        | Lambda |         
        +--------+         
             *             
             *             
             *      

<H4>The Two Below code works the same way !</H4>

In [23]:
SystemMessage(content="You are a nice pirate") + HumanMessage(content="What is Ramayan?")

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are a nice pirate', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is Ramayan?', additional_kwargs={}, response_metadata={})])

In [24]:
ChatPromptTemplate(messages=[
    SystemMessage(content="You are a nice pirate"),
    HumanMessage(content="What is Ramayan?")
])

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are a nice pirate', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is Ramayan?', additional_kwargs={}, response_metadata={})])